In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')


In [24]:
data_train = pd.read_csv("under_sample_data.csv")

In [53]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
import gc

In [54]:
class add_feature(BaseEstimator, TransformerMixin):
    def __init__(self,additional=1):
        self.additional = additional
    
    def fit(self,X,y=None):
        return self
        
    def transform(self,X):    
#        data["wkd"] = all_data.register_time.dt.weekday
        dt1=pd.to_datetime(X["register_time"])
        X["register_time"] = dt1
        X["wkd"] = dt1.dt.weekday
        X["day"] = dt1.dt.day
        X["hour"] = dt1.dt.hour
        X["day_in_year"] = dt1.dt.dayofyear
        X["days_spfest"] = X["register_time"].map(lambda x: (x - pd.to_datetime("2018-02-16")).days)
#         data =  data.merge(wkd_hour_df,on=["wkd","hour"],how = "left")
#         data["day"] = data.register_time.dt.day
#         data["month"] = data.register_time.dt.month
#         data["day_in_year"] = data.register_time.dt.dayofyear
#         data["days_spfest"] = data["register_time"].map(lambda x: (x - pd.to_datetime("2018-02-16")).days)

        X = X.drop(["register_time"],axis=1)
        if "prediction_pay_price" in X.columns:
            X = X.drop("prediction_pay_price",axis=1)
        X["price_per"] = (X["pay_price"]+0.1)/(X["pay_count"]+0.1)

        X["pvp_win_rate"] = (X.pvp_win_count + 0.1)/(X.pvp_lanch_count+0.1)
        X["pve_win_rate"] = (X.pve_win_count+0.1)/(X.pve_lanch_count+0.1)
        X["pvp_rate"] = (X.pvp_lanch_count+0.1)/(X.pve_lanch_count+0.1)
        X["pvp_vs_pve_win"] = (X.pvp_win_count+0.1)/(X.pve_win_count+0.1)
        
#         X = X.merge(hour_mean,on="hour",how = "left")
#         X = X.merge(wkd_mean,on="wkd",how="left")
#         X = X.merge(dayinyear_mean,how="left",on="day_in_year")
#         X = X.merge(dayinyear_hr_mean,how="left",on=["day_in_year","hour"])   
        X["avg_pay_price"] = X.pay_price/(X.avg_online_minutes+0.01)
        X["avg_pay_count"] = X.pay_count/(X.avg_online_minutes+0.01)
        X.loc[X.avg_online_minutes == 0,"avg_pay_price"] = 0
        X.loc[X.avg_online_minutes == 0,"avg_pay_count"] = 0
        X["build_remain"] = X.building_acceleration_add_value - X.building_acceleration_reduce_value
        X["build_use_rate"] = (X.building_acceleration_reduce_value + 0.1)/(X.building_acceleration_add_value + 0.1)
        X["treat_remain"] = X.treatment_acceleraion_add_value - X.treatment_acceleration_reduce_value
        X["treat_use_rate"] = (X.treatment_acceleration_reduce_value+0.1)/(X.treatment_acceleraion_add_value+0.1)
        X["res_remain"] = X.reaserch_acceleration_add_value - X.reaserch_acceleration_reduce_value
        X["res_use_rate"] = (X.reaserch_acceleration_reduce_value+ 0.1)/(X.reaserch_acceleration_add_value+ 0.1)
        #X.loc[X.avg_online_minutes == 0,"avg_acce_add"] = 0
        bd_cols_lv = [col for col in X.columns if col.startswith("bd")]
        sr_cols_lv = [col for col in X.columns if col.startswith("sr")]
        X["pve_vs_price"] = X["pve_win_count"]/(X["pay_price"]+0.1)
        X["pvp_vs_price"] = X["pvp_win_count"]/(X["pay_price"]+0.1)
        X["total_bd_lv"] = X[bd_cols_lv].sum(axis=1)
        X["total_sr_lv"] = X[sr_cols_lv].sum(axis=1)
        X["total_lv"] = X["total_bd_lv"] + X["total_sr_lv"]
        X["sr_prod_lv"] = X[['sr_rss_a_prod_levell','sr_rss_b_prod_level','sr_rss_c_prod_level','sr_rss_d_prod_level']].sum(axis=1)
        X["ivory_use_rate"] = (X["ivory_reduce_value"] + 0.1)/(X["ivory_add_value"] + 0.1)
        X["ivory_remain"] = X["ivory_add_value"] - X["ivory_reduce_value"]
        gc.collect()
        return X

In [44]:
# for col in data_train.columns:
#     if col.startswith("bd"):
data_train["pve_vs_price"] = data_train["pve_win_count"]/(data_train["pay_price"]+0.1)
print data_train["pve_vs_price"].head()

0    10.000000
1    62.980769
2     0.000000
3     1.442308
4     0.000000
Name: pve_vs_price, dtype: float64


In [55]:
pipe = Pipeline([
    ('add_feature', add_feature(additional=2))
    ])

data_pipe = pipe.fit_transform(data_train)

In [58]:
print data_pipe.iloc[:,109:].head()

   hour  day_in_year  days_spfest  pve_vs_price  price_per  pvp_win_rate  \
0    10           59           12     10.000000   1.000000      1.000000   
1    23           44           -3     62.980769   0.990476      1.112721   
2     4           48            1      0.000000   1.000000      1.000000   
3     1           50            3      1.442308   0.990476      1.000000   
4    21           36          -11      0.000000   1.000000      1.000000   

   pve_win_rate  pvp_rate  pvp_vs_pve_win  avg_pay_price      ...       \
0      1.000000  0.090909        0.090909       0.000000      ...        
1      0.977629  0.463087        0.527079       0.004478      ...        
2      1.000000  1.000000        1.000000       0.000000      ...        
3      1.000000  2.290323        2.290323       0.083627      ...        
4      1.000000  1.000000        1.000000       0.000000      ...        

   treat_use_rate  res_remain  res_use_rate  pvp_vs_price  total_bd_lv  \
0        1.000000       

In [ ]:
#这个不知道怎么用
# for i,re in dayinyear_hr_mean.iterrows():
#             day = re['day_in_year']
#             hr = re['hour']
#             tmp = X[(all_data.day_in_year == day) & (X.hour)]
#             all_data.loc[tmp.index,'day_hr_rank']=tmp.pay_price.rank(mothod = 'max',pct = True)